In [1]:
import requests
import pandas as pd
from multiprocessing import Pool
from progressbar import ProgressBar
from unidecode import unidecode

In [18]:
call = 'http://www.federalregister.gov/api/v1/articles.json?fields%5B%5D=raw_text_url&fields%5B%5D=title&fields%5B%5D=toc_subject&fields%5B%5D=topics&per_page=1000&order=oldest&conditions%5Bpublication_date%5D%5Bgte%5D=2013-01-01&conditions%5Bagencies%5D%5B%5D=commerce-department&conditions%5Bagencies%5D%5B%5D=state-department&conditions%5Bagencies%5D%5B%5D=treasury-department'
r = requests.get(call)

In [19]:
# bug where api only returns last 10,000 documents in spite of there being 10,877 within the search range
# fork api and correct

print r.json()['description']
print r.json()['count']
df = pd.DataFrame(r.json()['results'])

Documents published on or after 01/01/2013 and from Commerce Department, State Department, or Treasury Department
10877


In [20]:
while 'next_page_url' in r.json():
    next_page = r.json()['next_page_url']
    r = requests.get(next_page)
    print r
    df = pd.concat([df, pd.DataFrame(r.json()['results'])], ignore_index=True)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [27]:
def grab_text(url):
    r = requests.get(url)
    r.encoding = 'utf-8'
    text = r.text
    return text

In [ ]:
pool = Pool()
results = pool.map(grab_text, df.raw_text_url.tolist())
pool.close()

In [ ]:
df['raw_text'] = results

In [ ]:
df